# CID Demo — Coupling / Independence Diagnostic

This notebook demonstrates how CID detects **learnable coupling**
between two signals that may *appear* independent.

The goal is **diagnostic**, not prediction or causality.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

np.random.seed(42)


## Concept

If two signals are truly independent:

A model trained on signal B **should not** predict signal A
better than random guessing.

If accuracy rises above chance, this suggests **learnable coupling**.


In [ ]:
N = 3000

# Independent signals
A_ind = np.random.randn(N)
B_ind = np.random.randn(N)

# Coupled signals
noise = 0.3 * np.random.randn(N)
A_cpl = np.random.randn(N)
B_cpl = np.sign(A_cpl) + noise


In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(10, 6), sharex=True)

ax[0,0].plot(A_ind[:300])
ax[0,0].set_title("A (independent)")

ax[0,1].plot(B_ind[:300])
ax[0,1].set_title("B (independent)")

ax[1,0].plot(A_cpl[:300])
ax[1,0].set_title("A (coupled)")

ax[1,1].plot(B_cpl[:300])
ax[1,1].set_title("B (coupled)")

plt.tight_layout()
plt.show()


## Learnability Test

We convert signal A into a **binary direction target**:

- 1 if next value increases
- 0 otherwise

We then test whether signal B helps predict this direction.


In [ ]:
def cid_test(A, B):
    y = (np.diff(A) > 0).astype(int)
    X = B[:-1].reshape(-1, 1)

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.4, random_state=42
    )

    model = LogisticRegression(max_iter=1000)
    model.fit(X_train, y_train)

    preds = model.predict(X_test)
    return accuracy_score(y_test, preds)


In [ ]:
acc_ind = cid_test(A_ind, B_ind)
acc_cpl = cid_test(A_cpl, B_cpl)

acc_ind, acc_cpl


## Interpretation

- Accuracy ≈ 0.50 → independence
- Accuracy > 0.55 → possible coupling
- Accuracy > 0.60 → strong learnable dependence

CID does **not** imply causality.
It only detects **learnable structure leakage**.

## Conclusion

CID correctly fails to learn from independent signals,
and successfully detects coupling when structure exists.

This makes it useful for:
- leakage detection
- independence validation
- pre-model diagnostics
